# NYC MTA Turnstile Usage Dataset

Have a look at / download the following dataset :

- Go to http://web.mta.info/developers/turnstile.html
- This dataset shows entry & exit counter values for each turnstile-device in each station in the NYC Subway System.
- Note these aren’t counts per interval, but equivalent to an “odometer” reading for each device.
 

# Step 0 --- data download

Tool used: Firefox plugin --- DownThemAll!!!

Since the analytics and visulization tasks are based on 2013 data stats. I only downloaded the last day of 2011, all 2012 data, all 2013 data, and the first day of 2014.

# Step 1 --- raw data load